In [ ]:
from Datos import Datos
from EstrategiaParticionado import ValidacionSimple
from EstrategiaParticionado import ValidacionCruzada
from Clasificador import ClasificadorNaiveBayes,Clasificador
from Verificador import Verificador_GaussianNB, Verificador_Multinominal
from MatrizConfusion import MatrizConfusion

from tabulate import tabulate
import sys

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB


In [ ]:
"""
Apartado 1 => Particionado

Análisis de las dos estrategias de particionado propuestas: simple, y
cruzada, para los conjuntos propuestos: german y tic-tac-toe. El análisis
consiste en una descripción de los índices de train y test devueltos por
cada uno de los métodos de particionado, junto con un comentario
sobre las ventajas/desventajas de cada uno de ellos.
"""

In [ ]:
"""
En cuanto al metodo de VALIDACIÓN SIMPLE es el mas sencillo de implementar, puesto que se basa en una division 
aleatoria en dos grupos. Sin embargo, aún siendo el modo de validación más sencillo, trae consigo una serie de
inconvenientes:
    
    1) El ratio de error, es altamente variable dependiendo de las instancias del dataset escogido, para 
    entrenamiento, y test.
    
    2) Al segregar una serie de instancias para entrenamiento y test, estamos provocando que durante el entrenamiento
    el clasificador, no contemple todas las posibles situaciones de un contexto, por lo tanto esto produce
    una sobrestimación del ratio de error.
    
Por su parte el método de VALIDACIÓN CRUZADA es más complejo de implementar, puesto que se trata de un proceso 
iterativo. Éste, busca dividir todo el conjunto de datos en k grupos, de tal modo que, solo uno de esos bloques se usa 
como test, mientras que el resto de divisiones se usan como entrenamiento. Por cada iteración se turnan los "roles", de 
tal modo, que todas las subdivisiones son testadas y todas han sido usadas como entrenamiento. 

En un escenario, en el que el conjunto de datos es pequeño, es muy superior a validacón simple y otra ventaja 
respecto a validación simple es que, prueba y valida todas las instancias de datos. Por otra parte, podemos decir que
el ratio de error en este tipo de validacion es mucho más preciso y real, puesto que se entrena con prácticamente todo
el conjunto de datos maximizando asi el modelo sin sobrestimar en las predicciones. A su vez, es de gran
utilidad el que el ratio de error sea calculado como un promedio de las estimaciones de cada iteración.

Respecto los inconvenientes, principalmente encontramos uno, que es el coste computacional que requiere este método.
La identificación de bloques, el número de iteraciones y sobre todo que en el caso de usar un dataset 
excesivamente grande, debemos contemplar la posibilidad de que se vuelva algo lento y costoso (en comparación con
validación simple) y por tanto no sea tan útil como en otras circunstancias.
"""

In [ ]:
"""Cambiar y poner bonito, pero lo necesito para el 2"""

fileName = "ConjuntosDatos/tic-tac-toe.data"
datos_tic = Datos(fileName)
    
    # Probamos con 75 porciento y 10 iteraciones (Validacion Simple)
validacion_simple_tic = ValidacionSimple(75,10)
aux_simple_tic = validacion_simple_tic.creaParticiones(datos_tic)

    # Probamos con 10 k-iteraciones
validacion_cruzada_tic = ValidacionCruzada(6)
aux_cruzada_tic = validacion_cruzada_tic.creaParticiones(datos_tic)


fileName = "ConjuntosDatos/german.data"
datos_ger = Datos(fileName)

    # Probamos con 75 porciento y 10 iteraciones (Validacion Simple)
validacion_simple_ger = ValidacionSimple(75,10)
aux_simple_ger = validacion_simple_ger.creaParticiones(datos_ger)

# Probamos con 10 k-iteraciones
validacion_cruzada_ger = ValidacionCruzada(6)
aux_cruzada_ger = validacion_cruzada_ger.creaParticiones(datos_ger)

In [ ]:
"""
Apartado 2 => Naive Bayes

Tabla con los resultados de la ejecución para los conjuntos de datos
analizados (tic-tac-toe y german). Considerar los dos tipos de
particionado.
Los resultados se refieren a las tasas de error/acierto y deben incluirse
tanto con la corrección de Laplace como sin ella. Se debe incluir tanto
el promedio de error para las diferentes particiones como su desviación
típica. Es importante mostrar todos los resultados agrupados en una
tabla para facilitar su evaluación.
Breve análisis de los resultados anteriores.
"""

In [ ]:
"""
 Como se puede observar en las tablas generadas, se cumple lo dicho en el apartado anterior, siendo el error medio de la validación cruzada más bajo que el de la simple. De lo cual se peude deducir que la validación cruzada es más percisa.

 Además, cabe destacar las diferencias entre ejecutar la validación sin y con la corrección de Laplace. En estos datasets pequeños, no parece que se aprecie mucha diferencia, pero si nos fijamos en la validación cruzada, veremos que en 'German' sí que nos cambia, subiendo un poco la tasa de error medio. Esto puede ser causado por la generación de ejemplos que no se encuentran en el dataset que realiza la correcciónd de Laplace para eliminar valores nulos de las tablas de atributos.

"""

In [ ]:
# Creacion del clasificador
Clasificador = ClasificadorNaiveBayes()

#       Sin Laplace
#  TIC TAC TOE
# Validacion con validacion simple
media_error1, media_tp1, media_fp1, media_tn1, media_fn1 = Clasificador.validacion(validacion_simple_tic,datos_tic,False)

# Validacion con validacion cruzada
media_error2, media_tp2, media_fp2, media_tn2, media_fn2 = Clasificador.validacion(validacion_cruzada_tic,datos_tic,False)

#  GERMAN  
# Validacion con validacion simple
media_error3, media_tp3, media_fp3, media_tn3, media_fn3 = Clasificador.validacion(validacion_simple_ger,datos_ger,False)

# Validacion con validacion cruzada
media_error4, media_tp4, media_fp4, media_tn4, media_fn4 = Clasificador.validacion(validacion_cruzada_ger,datos_ger,False)

# Impresion de los resultados
resultados = [[round(media_error1, 3), round(media_error3, 3)], [round(media_error2, 3), 
round(media_error4, 3)]]

print("Sin Laplace")
print(tabulate(resultados, headers=['Tasa de error', 'Tic-Tac-Toe', 'German'], showindex=['Val. Simple', 'Val. Cruzada'], tablefmt='fancy_grid'))

#       Con Laplace
#  TIC TAC TOE
# Validacion con validacion simple
media_error1, media_tp1, media_fp1, media_tn1, media_fn1 = Clasificador.validacion(validacion_simple_tic,datos_tic,True)

# Validacion con validacion cruzada
media_error2, media_tp2, media_fp2, media_tn2, media_fn2 = Clasificador.validacion(validacion_cruzada_tic,datos_tic,True)

#  GERMAN  
# Validacion con validacion simple
media_error3, media_tp3, media_fp3, media_tn3, media_fn3 = Clasificador.validacion(validacion_simple_ger,datos_ger,True)

# Validacion con validacion cruzada
media_error4, media_tp4, media_fp4, media_tn4, media_fn4 = Clasificador.validacion(validacion_cruzada_ger,datos_ger,True)

# Impresion de los resultados
resultados = [[round(media_error1, 3), round(media_error3, 3)], [round(media_error2, 3), 
round(media_error4, 3)]]
print("Con Laplace")
print(tabulate(resultados, headers=['Tasa de error', 'Tic-Tac-Toe', 'German'], showindex=['Val. Simple', 'Val. Cruzada'], tablefmt='fancy_grid'))


In [ ]:
"""
Apartado 3 => Evaluación de hipótesis mediante Análisis ROC
Matriz de confusión y diagramas del clasificador en el espacio ROC
"""

In [ ]:
"""
En cuanto a la matriz de confusión, podemos observar que en ambos casos la clase positiva es mucho más abundante en el dataset que la negativa, por tanto los números de la primera fila son más altos.

En segundo lugar, cabe destacar que la cantidad de aciertos en ambas es bastante mayor que el de fallos,lo que se traduce en una predicción por encima de la diagonal normal.

"""

In [ ]:
# Calculamos la media de las tasas de val. simple y val. cruzada para la matriz de confusion media
    mx1 = MatrizConfusion()

    # TIC-TAC-TOE
    print("\nTic-Tac-Toe\n")
    tpr, fpr = mx1.matrix_media(media_tp1, media_tp2, media_fp1, media_fp2, 
                    media_tn1, media_tn2, media_fn1, media_fn2)
    plot_points = [[fpr, tpr, 'NB']]
    mx1.plot(plot_points, "tic-tac-toe")

    # GERMAN
    print("\nGerman\n")

    tpr, fpr = mx1.matrix_media(media_tp3, media_tp4, media_fp3, media_fp4, 
                    media_tn3, media_tn4, media_fn3, media_fn4)
    plot_points = [[fpr, tpr, 'NB']]
    mx1.plot(plot_points, "german")